In [ ]:
# pip install matplotlib
# pip install pyspark[sql]
# ipython package fo jupyter notebooks

# easy spark install using conda
# https://medium.com/the-ai-guide/easy-install-pyspark-in-anaconda-e2d427b3492f

# for installing java, scala for spark. actually don't use this
# https://medium.com/@aitmsi/single-node-spark-pyspark-cluster-on-windows-subsystem-for-linux-wsl2-22860888a98d
# maybe i should uninstall this
# $ sudo apt update && upgrade
# $ sudo apt-get install openjdk-11-jre
# $ sudo apt-get install openjdk-11-jdk

# for installing airflow
# https://sebastien-sime.medium.com/installing-apache-airflow-on-windows-10-wsl-314da1b28f3b
# added a /etc/wsl.conf file and filled it according to^
# pip3 install apache-airflow==1.10.14

In [2]:
import pyspark
# import matplotlib.pyplot as plt
from pyspark import SparkContext  # , SparkConf
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, LongType, FloatType, DateType
from pyspark.sql.functions import isnan, isnull, when, count, col

from airflow import DAG
from airflow.operators.python_operator import PythonOperator
# from airflow.hooks.postgres_hook import PostgresHook
# from airflow.hooks.S3_hook import S3Hook
# import pandas as pd
from io import StringIO
from datetime import datetime
import re
# import psycopg2p

/tmp/ipykernel_1673/3500185117.py:9 DeprecationWarning: The `airflow.operators.python_operator.PythonOperator` class is deprecated. Please use `'airflow.operators.python.PythonOperator'`.

In [5]:
# Creating a spark context class
# sc = SparkContext()

# Creating a spark session
spark = SparkSession.builder.appName('pySparkSetup').getOrCreate()

sc = spark.sparkContext

In [10]:
# Load data
DATA_DIR = "data"
COVID_ECONOMY_FILE = f"{DATA_DIR}/economy.csv"
COVID_EPIDEMIOLOGY_FILE = f"{DATA_DIR}/epidemiology.csv"
COVID_WEATHER_FILE = f"{DATA_DIR}/weather.csv"

In [35]:
# https://github.com/GoogleCloudPlatform/covid-19-open-data/blob/main/docs/table-economy.md

schema = StructType([
    StructField('location_key', StringType(), True),
    StructField('gdp_usd', LongType(), True),
    StructField('gdp_per_capita_usd', IntegerType(), True),
    StructField('human_capital_index', FloatType(), True)
])

economy_df = spark.read.format("csv").schema(schema).option("header", True).load(COVID_ECONOMY_FILE)

economy_df.limit(5).show()

+------------+------------+------------------+-------------------+
|location_key|     gdp_usd|gdp_per_capita_usd|human_capital_index|
+------------+------------+------------------+-------------------+
|          AD|  3154057987|             40886|               null|
|          AE|421142267937|             43103|              0.659|
|          AF| 19101353832|               502|              0.389|
|          AG|  1727759259|             17790|               null|
|          AL| 15278077446|              5352|              0.621|
+------------+------------+------------------+-------------------+



In [34]:
print("null count")
economy_df.select([count(when(isnull(c), c)).alias(c) for c in economy_df.columns]).show()
# print("nan count")  # none
# economy_df.select([count(when(isnan(c), c)).alias(c) for c in economy_df.columns]).show()

# Drop Nulls
cleaned_economy_df = economy_df.na.drop(subset=['gdp_usd', 'gdp_per_capita_usd'])

print("TODO: still need to handle human_capital_index")

print(f"Reduced {economy_df.count()} rows to {cleaned_economy_df.count()} rows")

print("Duplicates")  # none
duplicate_rows = cleaned_economy_df.count() - cleaned_economy_df.dropDuplicates().count()
print(f"Number of duplicate rows: {duplicate_rows}")
cleaned_economy_df = cleaned_economy_df.dropDuplicates()
cleaned_economy_df.limit(10).show()

null count
+------------+-------+------------------+-------------------+
|location_key|gdp_usd|gdp_per_capita_usd|human_capital_index|
+------------+-------+------------------+-------------------+
|           0|     31|                39|                248|
+------------+-------+------------------+-------------------+

TODO: still need to handle human_capital_index
Reduced 404 rows to 334 rows
Duplicates
Number of duplicate rows: 0
+------------+------------+------------------+-------------------+
|location_key|     gdp_usd|gdp_per_capita_usd|human_capital_index|
+------------+------------+------------------+-------------------+
|       DE_ST| 73969539000|             33394|               null|
|       IT_25|457916381400|             45548|               null|
|        AT_5| 34272274000|             61832|               null|
|       IT_88| 41212125400|             25016|               null|
|          GN| 13590281808|              1064|              0.374|
|      BE_BRU| 99104176200|

In [79]:
# https://github.com/GoogleCloudPlatform/covid-19-open-data/blob/main/docs/table-epidemiology.md

schema = StructType([
    StructField('date', DateType(), True),
    StructField('location_key', StringType(), True),
    StructField('new_confirmed', IntegerType(), True),
    StructField('new_deceased', IntegerType(), True),
    StructField('new_recovered', IntegerType(), True),
    StructField('new_tested', IntegerType(), True),
    StructField('cumulative_confirmed', IntegerType(), True),
    StructField('cumulative_deceased', IntegerType(), True),
    StructField('cumulative_recovered', IntegerType(), True),
    StructField('cumulative_tested', IntegerType(), True)
])

epidemiology_df = spark.read.format("csv").schema(schema).option("header", True).load(COVID_EPIDEMIOLOGY_FILE)

epidemiology_df.limit(5).show()

+----------+------------+-------------+------------+-------------+----------+--------------------+-------------------+--------------------+-----------------+
|      date|location_key|new_confirmed|new_deceased|new_recovered|new_tested|cumulative_confirmed|cumulative_deceased|cumulative_recovered|cumulative_tested|
+----------+------------+-------------+------------+-------------+----------+--------------------+-------------------+--------------------+-----------------+
|2020-01-01|          AD|            0|           0|         null|      null|                   0|                  0|                null|             null|
|2020-01-02|          AD|            0|           0|         null|      null|                   0|                  0|                null|             null|
|2020-01-03|          AD|            0|           0|         null|      null|                   0|                  0|                null|             null|
|2020-01-04|          AD|            0|           0|

In [80]:
df = epidemiology_df

print("null count")
df.select([count(when(isnull(c), c)).alias(c) for c in df.columns]).show()

# print("nan count")  # none
# df.select([count(when(isnan(c), c)).alias(c) for c in df.columns]).show()

# Drop Nulls
cleaned_df = df.na.drop()  # subset=['gdp_usd', 'gdp_per_capita_usd']


for col_name in ['new_confirmed', 'new_deceased', 'new_recovered', 'new_tested']:
    print(f"Changing {cleaned_df.where(col(col_name) < 0).select(col(col_name)).count()}, negative values in {col_name} to 0")

    cleaned_df = cleaned_df.withColumn(col_name, when(col(col_name) < 0, 0).otherwise(col(col_name)))

# print("Duplicates")  # none
duplicate_rows = cleaned_df.count() - cleaned_df.dropDuplicates().count()
print(f"Number of duplicate rows: {duplicate_rows}")
cleaned_df = cleaned_df.dropDuplicates()
cleaned_df.limit(10).show()
print(f"Reduced {df.count()} rows to {cleaned_df.count()} rows")
claned_epidemiology_df = cleaned_df

null count


+----+------------+-------------+------------+-------------+----------+--------------------+-------------------+--------------------+-----------------+
|date|location_key|new_confirmed|new_deceased|new_recovered|new_tested|cumulative_confirmed|cumulative_deceased|cumulative_recovered|cumulative_tested|
+----+------------+-------------+------------+-------------+----------+--------------------+-------------------+--------------------+-----------------+
|   0|           0|        50025|      858687|      8545363|   9331336|              198780|            1051000|             8534668|          9512906|
+----+------------+-------------+------------+-------------+----------+--------------------+-------------------+--------------------+-----------------+



Changing 26453, negative values in new_confirmed to 0


Changing 6986, negative values in new_deceased to 0


Changing 1888, negative values in new_recovered to 0


Changing 2769, negative values in new_tested to 0


Number of duplicate rows: 0


+----------+------------+-------------+------------+-------------+----------+--------------------+-------------------+--------------------+-----------------+
|      date|location_key|new_confirmed|new_deceased|new_recovered|new_tested|cumulative_confirmed|cumulative_deceased|cumulative_recovered|cumulative_tested|
+----------+------------+-------------+------------+-------------+----------+--------------------+-------------------+--------------------+-----------------+
|2020-12-23|          AT|         2843|          99|         3115|     38440|              344938|               6994|              307537|          3682136|
|2021-02-07|          AT|          993|          40|         1052|    212647|              420176|               9780|              391279|         10313170|
|2021-06-08|          AT|          356|           4|          607|    412011|              641360|              12964|              620881|         45983005|
|2021-08-15|          AT|          797|           1|

Reduced 12525825 rows to 2450433 rows


In [81]:
# df.cache()

In [78]:
# https://github.com/GoogleCloudPlatform/covid-19-open-data/blob/main/docs/table-weather.md

schema = StructType([
    StructField('date', DateType(), True),
    StructField('location_key', StringType(), True),
    StructField('average_temperature_celsius', FloatType(), True),
    StructField('minimum_temperature_celsius', FloatType(), True),
    StructField('maximum_temperature_celsius', FloatType(), True),
    StructField('rainfall_mm', FloatType(), True),
    StructField('snowfall_mm', FloatType(), True),
    StructField('dew_point', FloatType(), True),
    StructField('relative_humidity', FloatType(), True),
])

weather_df = spark.read.format("csv").schema(schema).option("header", True).load(COVID_WEATHER_FILE)  # 

weather_df.limit(5).show()

+----------+------------+---------------------------+---------------------------+---------------------------+-----------+-----------+---------+-----------------+
|      date|location_key|average_temperature_celsius|minimum_temperature_celsius|maximum_temperature_celsius|rainfall_mm|snowfall_mm|dew_point|relative_humidity|
+----------+------------+---------------------------+---------------------------+---------------------------+-----------+-----------+---------+-----------------+
|2020-01-01|          AD|                   4.236111|                   0.138889|                   8.208333|      3.302|       null|-0.972222|         72.77305|
|2020-01-02|          AD|                      3.875|                  -0.722222|                  10.055556|   6.688667|       null|   -1.625|         70.84132|
|2020-01-03|          AD|                   4.763889|                   0.597222|                   8.402778|     5.0165|       null|-0.611111|         71.11725|
|2020-01-04|          AD|   

In [70]:
!head -2 $COVID_WEATHER_FILE

date,location_key,average_temperature_celsius,minimum_temperature_celsius,maximum_temperature_celsius,rainfall_mm,snowfall_mm,dew_point,relative_humidity
2020-01-01,AD,4.236111,0.138889,8.208333,3.302,,-0.972222,72.773049
